Work Done as of Sunday Night:
-Still need to fix so that w can be calculated for uninclined orbits, look at homework 6 solutions, this is a prime example so see what formula is used!

-Something with dot product being positive or negative, and whether or not to add 2pi to O,w,and/or v. 



Taking Chris's advice, I need to update this code so that it can handle longer inputs depending on the decimal value. Newton's method fails for some level of this, so either want to switch to bi section method, or just add some dumb and over the top condition that influences how many data points I do for Newton's where it corresponds to the desired e of the orbit times the number of sig figs in the anomaly to be converted. 

3. The following program is due in four weeks (with HW #11).


Two-Body Problem. Write a two-part computer program that translates i) from orbital elements
(a, e, i, Ω, ω, ν) to positions and velocities (x, y, z, vx, vy, vz) and ii) from positions and
velocities back to orbital elements. 

Devise your own algorithms, or use the ones given in Danby,
Section 6.15. For Danby’s algorithm, note that Eq. 6.15.4 comes from 6.2.5 and that P is a
vector pointing from the mass-occupied focus to pericenter with magnitude e.


Basic Program Your program should work for 2D (i = Ω = 0) elliptical orbits, but does not
have to handle 3D or unbound (parabolic and hyperbolic) orbits.


Extra Credit Your program should work for all 3D bound orbits.


More Extra Credit Your program should handle hyperbolic and parabolic orbits as well as
elliptical ones.


Be thorough in testing your program! Test it by translating (x, y, z, vx, vy, vz) → (a, e, i, Ω, ω, ν) →
(x, y, z, vx, vy, vz) and (a, e, i, Ω, ω, ν) → (x, y, z, vx, vy, vz) → (a, e, i, Ω, ω, ν) for a number of
cases. This is why it is best to write both subroutines at the same time! 

Also test your programs
against your intuition for a number of special cases (e.g. circular orbits ought to have
r ⊥ v). 

Finally, compare your results with the “Changing the Elements” application in the
Working with Orbits section of the Astronomy Workshop. Do this comparison last, as in normal
coding situations you usually do not have an already-working example available to you!
Even More Extra Credit Can you find any non-trivial errors in the “Changing the Elements”
application? Trivial errors include round-off issues, etc.

Two-Body Problem. Write a two-part computer program that translates i) from orbital elements
(a, e, i, Ω, ω, ν) to positions and velocities (x, y, z, vx, vy, vz) and ii) from positions and
velocities back to orbital elements. 


i) create OE2cart(a,e,i,Ω, ω, ν):

In [1]:
import numpy as np
import math as mth

In [2]:
def norm(x):
    xnorm = []
    mag = 0
    for i in range(len(x)):
        mag += x[i]**2    
    for j in range(len(x)):
        xnorm.append(x[j]/np.sqrt(mag))
        
    return xnorm

def mag(x):
    #get the magnitude of a vector
    value = 0
    for i in range(len(x)):
        value += x[i]**2
    return np.sqrt(value)

def angle(v1, v2):

    return mth.acos(np.dot(v1, v2) / (mag(v1) * mag(v2)))

In [5]:
def OE2cart(a,e,i,O,w,v):
    #first to put it in terms of radians...
    i = np.deg2rad(i)    
    O = np.deg2rad(O)
    w = np.deg2rad(w)            
    v = np.deg2rad(v)

    GM=1
    p=a*(1-e**2)
#start with work in a perifocal coordinate frame... 
    r=p/(1+e*np.cos(v))
    #magnitude of r right here...
    #now magnitude of v...
#v=sqrt(GM/p)[-sinvX+(e+cosv)Y]

    velperifocal=np.sqrt(GM*(2/r-1/a)) 
    
    rP=r*np.cos(v)
    rQ=r*np.sin(v)
    rperifocal=[[rP],[rQ],[0]]
    vP=np.sqrt(GM/p)*(-np.sin(v))
    vQ=np.sqrt(GM/p)*(e+np.cos(v))
    vperifocal = [[vP],[vQ],[0]]
    #obtained from Fundamental of Astrodynamics, using a matrix transformation from perifocal to Cartesion coord systems. 
    R11=(np.cos(O)*np.cos(w)-np.sin(O)*np.sin(w)*np.sin(i))
    R12 = -np.cos(O)*np.sin(w)-np.sin(O)*np.cos(w)*np.cos(i)
    R13 = np.sin(O)*np.sin(i)
    R21 = np.sin(O)*np.cos(w) + np.cos(O)*np.sin(w)*np.cos(i)
    R22 = -np.sin(O)*np.sin(w) + np.cos(O)*np.cos(w)*np.cos(i)
    R23 = -np.cos(O)*np.sin(i)
    R31 = np.sin(w)*np.sin(i)
    R32 = np.cos(w)*np.sin(i)
    R33 = np.cos(i)
    R=[[R11,R12,R13],[R21,R22,R23],[R31,R32,R33]]
     
    rcart=[[0],[0],[0]]
    vcart=[[0],[0],[0]]
    for i in range(len(R)):
         for j in range(len(rperifocal[0])):
            for k in range(len(rperifocal)):
                rcart[i][j] += R[i][k] * rperifocal[k][j]
                vcart[i][j] +=R[i][k] *vperifocal[k][j]
    print("x,y,z,vx,vy,vz")
    return rcart,vcart

In [6]:
OE2cart(.64,.78,0,0,69,174.5)

x,y,z,vx,vy,vz


([[-0.50014489886025959], [-1.0031354762822711], [0.0]],
 [[0.3330678482478564], [-0.3329263917360944], [0.0]])

In [ ]:
OE2cart(1,.21,0,0,0,0)

In [ ]:
def cart2OE(x,y,z,vx,vy,vz):
    #first the things that will be used later... 
    GM=1
    r=np.array([x,y,z])
    rmag = mag(r)
    v=np.array([vx,vy,vz])
    vmag = mag(v)
    h=np.cross(r,v)
    I = np.array([1,0,0])  #x axis vector 
    K = np.array([0,0,1])  #z axis of this setup
    n = np.cross(K,h) #node vector, should be zero when uninclined
    nmag = mag(n)  #magnitude of node vector, use this to decide if equatorial orbit or not. 
 #   print("n " + str(nmag))
    #a
    a = 1/((2/rmag)-(vmag**2/GM))
    #e
    evec=np.cross(v,h)/GM-norm(r) #units in terms of circular velocity!
   # print("evec " + str(evec)) 
#   print("normal r " + str(norm(r)))
    e=mag(evec)
    #i
    hmag = mag(h)
    hnorm = norm(h)
    i=np.arccos(hnorm[2])  #normalized w z axis, distance btw... seems to pass every test so far!
    #O
    if nmag>0:
        O = np.arccos(np.dot(n,I)/mag(n))
    else:
        O = 0
  #  print(n,I,np.dot(n,I))
    if nmag>0:     
        if evec[2] <0: 
            w = np.arccos(np.dot(n,evec)/(mag(evec)*mag(n))) 
        else: 
            w  = 2*np.pi + np.arccos(np.dot(n,evec)/(mag(evec)*mag(n)))
    else: 
        if h[2]<0:
            w = 2*np.pi - np.arctan2(evec[1],evec[0])  #formula given on wiki, 2D case. 
            if w < 
            # w = np.arccos(np.dot(evec,I)/mag(evec))
        else:
            w = 2*np.pi + np.arctan2(evec[1],evec[0])
            if w > 2*np.pi: #just so it doesn't go over domain
                w = w-2*np.pi
            #w = np.arccos(np.dot(evec,I)/mag(evec))


    p = a*(1-e**2)
#semi latus rectum, same definition as in other part 
    sep = 2*np.pi - angle(r,evec)
  # if 
   #     v = np.arccos((1/e)*((p/rmag)-1))
    
  #  v = np.arccos(np.dot(r,evec)/(mag(evec)*mag(r)))
   # print("dot product v " + str(np.dot(r,evec)))
    #print("angle " + str(np.rad2deg(angle(r,evec))))
    print("h = " + str(h))
    print("e = " + str(evec))
    print("a,e,i,O,w,v")
    return a,e,np.rad2deg(i),np.rad2deg(O),np.rad2deg(w),np.rad2deg(sep)

In [ ]:
cart2OE(-.5,-1,0,.3333,-.3333,0)

In order to determine what range the true anomaly is in, I'm going to have to figure out what side of the orbit it's in. By this, I mean it is either travelling away from pericenter (arccos is fine) or travelling towards v (arccos + 180) so I need a way to figure out which one it is. One way is to take the dot product between r and pericenter which is e... gonna test this now. 



In [ ]:
59.879781692586391 -300

In [ ]:
np.array([1,2,3])

In [ ]:
type([1,3,2])

In [ ]:
a=np.array([0,0,3])
b=np.array([0,0,3])

In [ ]:
np.cross(a,b)

In [ ]:
b=np.array([1,2,60])

In [ ]:
np.linalg.norm(b)

In [ ]:
def norm(x):
    xnorm = []
    mag = 0
    for i in range(len(x)):
        mag += x[i]**2    
    for j in range(len(x)):
        xnorm.append(x[j]/np.sqrt(mag))
        
    return xnorm

def mag(x):
    #get the magnitude of a vector
    value = 0
    for i in range(len(x)):
        value += x[i]**2
    return np.sqrt(value)

In [ ]:
b

In [ ]:
norm(b)

In [ ]:
a = [1,0,0]  #pericenter @ 1 
b = [.707,.707,0]

In [ ]:
np.dot(a,b)

In [12]:
def length(v):
    return mth.sqrt(np.dot(v, v))

def angle(v1, v2):

    return mth.acos(np.dot(v1, v2) / (length(v1) * length(v2)))

def normalize(x):
    xnorm = []
    mag = 0
    for i in range(len(x)):
        mag += x[i]**2    
    for j in range(len(x)):
        xnorm.append(x[j]/np.sqrt(mag))
        
    return xnorm

In [69]:
def cart2OE(x,y,z,vx,vy,vz):
    #first the things that will be used later... 
    GM=1
    r=np.array([x,y,z])
    rmag = length(r)
    v=np.array([vx,vy,vz])
    vmag = length(v)
    h=np.cross(r,v)
    I = np.array([1,0,0])  #x axis vector 
    K = np.array([0,0,1])  #z axis of this setup
    n = np.cross(K,h) #node vector, should be zero when uninclined
    nmag = length(n)  #magnitude of node vector, use this to decide if equatorial orbit or not. 
    #a
    a = 1/((2/rmag)-(vmag**2/GM))
    #e
    evec=np.cross(v,h)/GM-normalize(r) 
    e=mag(evec)
    #i
    hmag = mag(h)
    hnorm = norm(h)
    i=np.arccos(hnorm[2])  #normalized w z axis, distance btw... seems to pass every test so far!
    #O
    if i>0 and i<np.pi:
        if n[1]>=0:
            O = np.arccos(np.dot(n,I)/length(n))
        elif n[1]<0:
            O = 2*np.pi - np.arccos(np.dot(n,I)/length(n))
    else:
        O = 0
    #w
    if i > 0 and i < np.pi: 
        if evec[2]>=0:
            w = np.arccos(np.dot(n,evec)/(length(evec)*length(n)))
        elif evec[2]<0:
            w = 2*np.pi - np.arccos(np.dot(n,evec)/(length(evec)*length(n)))
    else:
        if h[2]>=0:
            w = 2*np.pi + np.arctan2(evec[1],evec[0])
            if w >= 2*np.pi:
                w = w - 2*np.pi
        else: 
            w = 2*np.pi - np.arctan2(evec[1],evec[0])
            if w >=2*np.pi:
                w = w - 2*np.pi
    #v
    if np.dot(r,v)>0:
        v = np.arccos(np.dot(evec,r)/(length(evec)*length(r)))
    elif np.dot(r,v)<0:
        v = 2*np.pi - np.arccos(np.dot(evec,r)/(length(evec)*length(r)))
    
  #  print("h = " + str(h))
    return a,e,np.rad2deg(i),np.rad2deg(O),np.rad2deg(w),np.rad2deg(v)

In [70]:
aa,ee,ii,OO,ww,vv = cart2OE(-.29,.118,0,-.471,-2.23,0)

In [71]:
OE2cart(aa,ee,ii,OO,ww,vv)

x,y,z,vx,vy,vz


([[-0.29000000000000009], [0.1180000000000004], [0.0]],
 [[-0.47100000000000164], [-2.2299999999999991], [0.0]])

In [62]:
cart2OE(-0.88,-.181,0,-0.797,0.635,0)

h = [-0.        0.       -0.703057]


(0.841968516670687, 0.64260120512335595, 180.0, 0.0, 33.949887636832656)

In [56]:
cart2OE(-0.172,0.103,0,-0.703,-0.7377,0)

h = [ 0.         0.         0.1992934]


(0.11188746512797258, 0.80313111054031072, 0.0, 0.0, 332.27343347920123)